In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Load your data
data_df = pd.read_csv('../Data/spam_utf8.csv')
train_texts = data_df['v2'].tolist()

# Load the TensorFlow Lite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='../models/spam.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

2024-04-15 09:31:14.685081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-15 09:31:14.685109: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
# Define parameters
max_features = 2000
sequence_length = 16

# Create the TextVectorization layer
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

# Adapt the TextVectorization layer to the train texts
vectorize_layer.adapt(train_texts)

def preprocess_text(text):
    text = tf.expand_dims(text, -1)  # Reshape text to fit the vectorization layer input
    processed_text = vectorize_layer(text)
    return np.array(processed_text, dtype=np.int32)  # Convert to numpy array with type INT32


2024-04-15 09:31:17.819438: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-04-15 09:31:17.819465: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: dewgong
2024-04-15 09:31:17.819470: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: dewgong
2024-04-15 09:31:17.819625: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.67.0
2024-04-15 09:31:17.819641: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.67.0
2024-04-15 09:31:17.819645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 550.67.0
2024-04-15 09:31:17.820215: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical oper

In [3]:
def spam_detection(message, model_interpreter):
    try:
        processed_message = preprocess_text(message)
        model_interpreter.set_tensor(input_details[0]['index'], processed_message)
        model_interpreter.invoke()
        output_data = model_interpreter.get_tensor(output_details[0]['index'])
        probabilities = output_data[0]
        is_spam = probabilities[1] > 0.5
        print(f"Message: {message}\nProbabilities: Not Spam = {probabilities[0]:.5f}, Spam = {probabilities[1]:.5f}\nIs Spam: {is_spam}\n")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
non_spam_message = "How’s everything going? Mom mentioned you were feeling under the weather."
print("Non Spam Example: ")
spam_detection(non_spam_message, interpreter)

print("Spam Example: ")
spam_message = "Warning!!! You have three viruses on your device! Click here immediately for free virus removal: http://malware-site.com"
spam_detection(spam_message, interpreter)

Non Spam Example: 
Message: How’s everything going? Mom mentioned you were feeling under the weather.
Probabilities: Not Spam = 0.99824, Spam = 0.00176
Is Spam: False

Spam Example: 
Message: Warning!!! You have three viruses on your device! Click here immediately for free virus removal: http://malware-site.com
Probabilities: Not Spam = 0.00018, Spam = 0.99982
Is Spam: True

